In [ ]:
from google.colab import drive

# Unmount the drive first
drive.flush_and_unmount()
print('Drive unmounted')

# Remove existing files from the mountpoint if it exists
import os
if os.path.exists('/content/drive'):
  !rm -rf '/content/drive'  # Use with caution! This permanently deletes all files in the directory.
  print('Files removed from mountpoint')

# Remount the drive
drive.mount('/content/drive')
print('Drive mounted')

Getting the songs:

In [ ]:
#!pip install yt-dlp
import yt_dlp
import os

def download_audio_from_youtube_playlist(playlist_url, genre, output_base_folder="Downloaded_Audio"):
    # Ensure the base output folder exists
    if not os.path.exists(output_base_folder):
        os.makedirs(output_base_folder)

    # Create a subfolder for each genre within the base folder
    output_folder = os.path.join(output_base_folder, genre)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # yt-dlp options for downloading audio
    ydl_opts = {
        'format': 'bestaudio/best',  # Best available audio-only format
        'outtmpl': f'{output_folder}/%(title)s.%(ext)s',  # Save file naming template
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # Convert audio to WAV format
            'preferredquality': '192',  # Optional: Set audio quality (doesn't affect WAV as much)
        }],
        'ignoreerrors': True,  # Skip videos with errors
        'retries': 3,  # Retry up to 3 times in case of intermittent errors
    }

    # Download the playlist
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            print(f"Downloading playlist from URL: {playlist_url}")
            ydl.download([playlist_url])
            print(f"Download complete for playlist: {playlist_url}")
        except yt_dlp.DownloadError as e:
            print(f"An error occurred: {e}")
            print("Some videos in the playlist may be unavailable or restricted.")

# List of playlist URLs with corresponding genres
playlists = [
    {"url": "https://www.youtube.com/playlist?list=PLFjmckBbDlzSE47xD0FivigDOARiLJl61", "genre": "Blues"},
    {"url": "https://www.youtube.com/playlist?list=PL2788304DC59DBEB4", "genre": "Classical"},
    {"url": "https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S", "genre": "Country"},
    {"url": "https://www.youtube.com/playlist?list=PL-Ib9oOPR7OHKLBFVkiq0F0rppCZ7YFLp", "genre": "Jazz"},
    {"url": "https://www.youtube.com/playlist?list=PLmXxqSJJq-yUwqtbp8MHBoTDoDULMoViq", "genre": "Metal"},
    {"url": "https://www.youtube.com/playlist?list=PLxA687tYuMWhkqYjvAGtW_heiEL4Hk_Lx", "genre": "Pop"},
    {"url": "https://www.youtube.com/playlist?list=PLEXox2R2RxZKUmrWKNF61K-kZSov14Snr", "genre": "Disco"},
    {"url": "https://youtube.com/playlist?list=PLvuMfxvpAQrkzez9insKS8cGPU74sK1Ss&si=xKGfXOnrRSZ9CGiG", "genre": "Hip-Hop"},
    {"url": "https://youtube.com/playlist?list=PLw7aLrPJ8Hl24kczxvRoPECfTvAMsOwv1&si=Xt0CIMq-JXp3rLpQ", "genre": "Reggae"},
    {"url": "https://youtube.com/playlist?list=PLVQ7g3e6O27cH8KG9mktLWH8zcqiwTntP&si=u40PwF1n7IGzaQZj", "genre": "Rock"},
    {"url": "https://youtube.com/playlist?list=PLBD52290539D96397&si=pHtbrcvxXbOHGZAc", "genre": "Rap"},
    {"url": "https://youtube.com/playlist?list=PL2z0emdmB6sCiM865_tVZIv8Ut_9mxqaq&si=4ZOlLO4w_JzZcvw4", "genre": "K-Pop"},
    {"url": "https://youtube.com/playlist?list=PL5PiyRsNH6iolTZhrMmOxfdQocnokwdpX&si=z6n3stJkKrv7TYpX", "genre": "Alternative"},
    ]

# Specify the base folder to save all downloaded playlists
output_base_folder = "/content/drive/Shareddrives/Machine_Learning_Project_Drive/Youtube_Data"

# Loop through each playlist and genre
for playlist in playlists:
    download_audio_from_youtube_playlist(playlist["url"], playlist["genre"], output_base_folder)


Creating the CSV:

In [ ]:
import librosa
import pandas as pd
import os

def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, sr=None)
        audio, _ = librosa.effects.trim(audio)

        # Extracting features
        chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        rms = librosa.feature.rms(y=audio)
        spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sample_rate)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
        zero_crossings = librosa.zero_crossings(audio, pad=False)
        y_harm, y_perc = librosa.effects.hpss(audio)
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sample_rate)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sample_rate)

        features = {
            #fix ordering and filename should be song
            'zero_crossing_rate_mean': sum(zero_crossings) / len(zero_crossings),
            'sample_rate': sample_rate,
            'genre': file_name.split('/')[-2],
            'song': file_name.split('/')[-1],
            'length': len(audio),
            'chroma_stft_mean': chroma_stft.mean(),
            'chroma_stft_var': chroma_stft.var(),
            'rms_mean': rms.mean(),
            'rms_var': rms.var(),
            'spectral_centroid_mean': spectral_centroid.mean(),
            'spectral_centroid_var': spectral_centroid.var(),
            'spectral_bandwidth_mean': spectral_bandwidth.mean(),
            'spectral_bandwidth_var': spectral_bandwidth.var(),
            'spectral_rolloff_mean': spectral_rolloff.mean(),
            'y_harm_mean': y_harm.mean(),
            'y_perc_mean': y_perc.mean(),
            'tempo': tempo,
        }

        for i in range(1, 21):
            features[f'mfcc{i}_mean'] = mfccs[i-1].mean()
            features[f'mfcc{i}_var'] = mfccs[i-1].var()

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name} - {e}")
        return None

def save_features_to_csv(file_list, output_csv):
    features_list = []
    for file_name in file_list:
        features = extract_features(file_name)
        if features is not None:
            features_list.append(features)

    df = pd.DataFrame(features_list)
    df.to_csv(output_csv, index=False)

audio_files = []
general_path = "/content/drive/Shareddrives/Machine_Learning_Project_Drive/Youtube_Data"

for genre in os.listdir(general_path):
    if genre == ".ipynb_checkpoints":
        continue
    for song in os.listdir(f'{general_path}/{genre}'):
        audio_files.append(f'{general_path}/{genre}/{song}')

output_csv = f'{general_path[:-len("/Youtube_Data")]}/audio_features.csv'
save_features_to_csv(audio_files, output_csv)

In [ ]:
fixing format
data = pd.read_csv('/content/drive/Shareddrives/Machine_Learning_Project_Drive/Audio_Features/all_audio_features.csv')
data['tempo'] = [float(x.strip('[]')) for x in data['tempo']]
data.to_csv('/content/drive/Shareddrives/Machine_Learning_Project_Drive/Audio_Features/all_audio_features_modified.csv', index=False)
